<a href="https://colab.research.google.com/github/kasivisu4/NEU_CLASS/blob/main/Algorithms_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("HI");

HI


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
pip install anytree

     |████████████████████████████████| 41 kB 226 kB/s 


In [9]:
import csv
from anytree import Node, RenderTree

In [6]:
with open('/content/drive/My Drive/Colab Notebooks/UNSPSC_English.csv',encoding="utf8") as unspsc_file:
    [ next(unspsc_file) for _ in range(9)]  
    unspsc_reader = csv.DictReader(unspsc_file, delimiter=',')
    titles=[["Segment","Segment Title"],["family","Family Title"],["Class","Class Title"],["Commodity","Commodity Title"]]
    for line in unspsc_reader:
      print(line)
      break


OrderedDict([('Version', 'UNSPSC v23_0701'), ('Key', '100001'), ('Segment', '10000000'), ('Segment Title', 'Live Plant and Animal Material and Accessories and Supplies'), ('Segment Definition', 'This segment includes live, wild and domesticated, seeds and plants and animals. This segment also includes materials and equipment used in the care, containment and feeding of live plants and animals.'), ('Family', ''), ('Family Title', ''), ('Family Definition', ''), ('Column1', ''), ('Class', ''), ('Class Title', ''), ('Class Definition', ''), ('Commodity', ''), ('Commodity Title', ''), ('Definition', ''), ('Synonym', '活植物、活动物及其附件和用品, 活植物、活動物及其附件和用品, Levende og animalsk materiale, tilbehør og forsyning, Levende planten &amp; dieren materialen. Toebehoren &amp; toeleveringen, Fournitures, accessoires et matériel pour plantes et animaux vivants, Lebende Pflanzen und Tiermaterial und Zubehör und Bedarf, Élő növényi és állati eredetű anyagok, tartozékok és felszerelések, Piante e Animali, Access

In [ ]:
def create_Node(data):
    if not check_Parent(data["segment_title"]):
        nodes[data["segment_title"]] = Node(data["segment_title"],parent=root,data=data)
    else:
        nodes[data["family"]]
    

def check_Parent(data_title):
    if data_title in nodes:
        return True
    return False
    